In [1]:
from pyspark.streaming import StreamingContext

# Create a StreamingContext with batch interval of 5 seconds
ssc = StreamingContext(sc, 5)

# Create a DStream that will connect to localhost at port 9999
# Start Netcat server: nc -lk 9999 
lines = ssc.socketTextStream('localhost', 9999)

# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
lines.pprint()
wordCounts.pprint()

ssc.start()  # Start the computation
print ("Start")
ssc.awaitTermination(20)  # Wait for the computation to terminate
ssc.stop(stopSparkContext=False)  # Stop the StreamingContext without stopping the SparkContext

print ("Finished")

Start
-------------------------------------------
Time: 2018-11-16 21:58:40
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:40
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:45
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:45
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:50
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:50
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:55
-------------------------------------------

-------------------------------------------
Time: 2018-11-16 21:58:55
-------------------------------------------

Finished


In [12]:
'''
Create a queue of 10 RDDs using this data set and 
feed it into a Spark Streaming program.

Your Spark Streaming algorithm should maintain a 
state that keeps track of the longest noun seen so 
far associated with each distinct adjective. 

After each RDD, print any 5 adjectives and their 
associated longest nouns, as well as the longest 
noun associated with the adjective 'good'.


 Note that not every line in the data set contains 
 exactly two words, so make sure to clean the data 
 as they are fed into the streaming program.  
 The skeleton code is provided below:

'''

from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
noun_lens = pairs.map(lambda m: (m[0], (m[1],len(m[1])))).reduceByKey(lambda x, y: x if x[1] > y[1] else y)
longest_nouns = noun_lens.transform(lambda rdd: rdd.sortBy(lambda x: x[1][1], False))

def printResults(rdd):
    print (rdd.take(5))
    print ('good:', rdd.lookup('good'))

longest_nouns.foreachRDD(printResults)

# FILL IN YOUR CODE

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

[('medical', ('pneumonoultramicroscopicsilicovolcanoconiosis', 45)), ('result', ('ortho-nitro-para-diamino-triphenylmethane', 41)), ('comma-free', ('00101110100001100010111010000110', 32)), ('at-taqafah', ('wad-dimuqratiyyah/rassemblement', 31)), ('delete', ('atlasshrugged/passengernumber4', 30))]
good: [('morning/afternoon', 17)]
[('detective-ghost-horror-who', ('travel-romantic-musical-comedy-epic', 35)), ('dunnit-time', ('travel-romantic-musical-comedy-epic', 35)), ('Upper', ('famennian/chautauquan/canadaway', 31)), ('vertical', ('encodingssynchronizationvideo', 29)), ('encyclopedia_list_links_solicited', ('building_wikipedia_membership', 29))]
good: [('morning/afternoon', 17)]
[('natural', ('5-aminoimidazole-4-carboxamide-1-ß-d-ribofuranoside', 51)), ('automatic', ('transmission/gearbox/transmission', 33)), ('such', ('quarantine-for-decontamination', 30)), ('delete', ('HoldMoreStubbornlyAtLeastTalk', 29)), ('legal', ('implications/interpretations', 28))]
good: [('passage-planning',

In [7]:
from pyspark.streaming import StreamingContext

# Create a queue of RDDs
rdd = sc.textFile('adj_noun_pairs.txt', 8)

# split the rdd into 5 equal-size parts 
# random seed
rddQueue = rdd.randomSplit([1,1,1,1,1], 123)
        
# Create a StreamingContext with batch interval of 5 seconds
ssc = StreamingContext(sc, 5)

# Feed the rdd queue to a DStream
lines = ssc.queueStream(rddQueue)

# Do word-counting as before
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Use transform() to access any rdd transformations not directly available in SparkStreaming
topWords = wordCounts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))
topWords.pprint()

ssc.start()  # Start the computation
ssc.awaitTermination(25)  # Wait for the computation to terminate
ssc.stop(False)
print("Finished")

-------------------------------------------
Time: 2018-11-16 22:15:35
-------------------------------------------
('other', 15486)
('first', 10815)
('many', 9773)
('new', 6272)
('system', 5063)
('american', 4744)
('several', 4545)
('century', 4492)
('same', 4394)
('=', 4313)
...

-------------------------------------------
Time: 2018-11-16 22:15:40
-------------------------------------------
('other', 15319)
('first', 10709)
('many', 9575)
('new', 6242)
('system', 5111)
('american', 4777)
('century', 4538)
('several', 4515)
('same', 4453)
('=', 4361)
...

-------------------------------------------
Time: 2018-11-16 22:15:45
-------------------------------------------
('other', 15346)
('first', 10517)
('many', 9706)
('new', 6218)
('system', 5266)
('american', 4940)
('several', 4615)
('=', 4451)
('century', 4438)
('same', 4270)
...

-------------------------------------------
Time: 2018-11-16 22:15:50
-------------------------------------------
('other', 15196)
('first', 10617)
('many', 

In [4]:
# Find the most positive words in windows of 5 seconds from streaming data

from pyspark.streaming import StreamingContext

def parse_line(l):
    x = l.split("\t")
    return (x[0], float(x[1]))

word_sentiments = sc.textFile("AFINN-111.txt") \
                    .map(parse_line).cache()
    
ssc = StreamingContext(sc, 5)
rdd = sc.textFile('adj_noun_pairs.txt', 8)
rddQueue = rdd.randomSplit([1,1,1,1,1], 123)
lines = ssc.queueStream(rddQueue)

word_counts = lines.flatMap(lambda line: line.split(" ")) \
                   .map(lambda word: (word, 1)) \
                   .reduceByKey(lambda a, b: a + b)

# Determine the words with the highest sentiment values by joining the streaming RDD
# with the static RDD inside the transform() method and then multiplying
# the frequency of the words by its sentiment value
happiest_words = word_counts.transform(lambda rdd: word_sentiments.join(rdd))\
   .map(lambda (word, tuple):(tuple[0] * tuple[1], word))\
   .transform(lambda rdd: rdd.sortByKey(False))

happiest_words.pprint()

ssc.start()
ssc.awaitTermination(25)
ssc.stop(False)
print("Finished")

SyntaxError: invalid syntax (<ipython-input-4-cde85e897556>, line 21)

In [3]:
from pyspark.streaming import StreamingContext

# Stateful word count

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory.  Required for stateful transformations
ssc.checkpoint("checkpoint")

rdd = sc.textFile('adj_noun_pairs.txt', 8)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)
    # add the new values with the previous running count to get the new count

running_counts = lines.flatMap(lambda line: line.split(" "))\
                      .map(lambda word: (word, 1))\
                      .reduceByKey(lambda a, b: a + b) \
                      .updateStateByKey(updateFunc)

counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))

def printResults(rdd):
    print ("Total distinct words: ", rdd.count())
    print (rdd.take(5))
    print ('refinery:', rdd.lookup('refinery')[0])

counts_sorted.foreachRDD(printResults)

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

Total distinct words:  51430
[('other', 7782), ('first', 5404), ('many', 4878), ('new', 3219), ('system', 2539)]
refinery: 1
Total distinct words:  76917
[('other', 15486), ('first', 10815), ('many', 9773), ('new', 6272), ('system', 5063)]
refinery: 6
Total distinct words:  97338
[('other', 23129), ('first', 16145), ('many', 14534), ('new', 9363), ('system', 7636)]
refinery: 11
Total distinct words:  114786
[('other', 30805), ('first', 21524), ('many', 19348), ('new', 12514), ('system', 10174)]
refinery: 16
Total distinct words:  130393
[('other', 38452), ('first', 26792), ('many', 24239), ('new', 15618), ('system', 12777)]
refinery: 19
Total distinct words:  144898
[('other', 46151), ('first', 32041), ('many', 29054), ('new', 18732), ('system', 15440)]
refinery: 23
Total distinct words:  158013
[('other', 53728), ('first', 37395), ('many', 33933), ('new', 21787), ('system', 17946)]
refinery: 26
Total distinct words:  170610
[('other', 61347), ('first', 42658), ('many', 38902), ('new',

In [17]:
# MG algorithm for approximate word count

from pyspark.streaming import StreamingContext

k = 10000
threshold = 0
total_decrement = 0

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory.  Required for stateful transformations
ssc.checkpoint("checkpoint")

rdd = sc.textFile('adj_noun_pairs.txt', 8)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    newValue = sum(newValues, runningCount) - threshold
    return newValue if newValue > 0 else None
    # add the new values with the previous running count to get the new count

running_counts = lines.flatMap(lambda line: line.split(" "))\
                      .map(lambda word: (word, 1))\
                      .reduceByKey(lambda a, b: a + b) \
                      .updateStateByKey(updateFunc)
            
counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))

def printResults(rdd):
    global threshold, total_decrement 
    rdd.cache()
    print ("Total distinct words: ", rdd.count())
    print (rdd.map(lambda x: (x[0], x[1], x[1]+total_decrement)).take(5))
    lower_bound = rdd.lookup('refinery')
    if len(lower_bound) > 0:
        lower_bound = lower_bound[0]
    else:
        lower_bound = 0
    print ('refinery:', lower_bound, ',', lower_bound + total_decrement)
    if rdd.count() > k:
        threshold = rdd.zipWithIndex().map(lambda x: (x[1], x[0])).lookup(k)[0][1]
    else:
        threhold = 0
    print ("Next threshold = ", threshold)
    total_decrement += threshold
    rdd.unpersist()

counts_sorted.foreachRDD(printResults)

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

Total distinct words:  51430
[('other', 7782, 7782), ('first', 5404, 5404), ('many', 4878, 4878), ('new', 3219, 3219), ('system', 2539, 2539)]
refinery: 1 , 1
Next threshold =  5
Total distinct words:  76917
[('other', 15486, 15491), ('first', 10815, 10820), ('many', 9773, 9778), ('new', 6272, 6277), ('system', 5063, 5068)]
refinery: 6 , 11
Next threshold =  10
Total distinct words:  17602
[('other', 23124, 23139), ('first', 16140, 16155), ('many', 14529, 14544), ('new', 9358, 9373), ('system', 7631, 7646)]
refinery: 6 , 21
Next threshold =  9
Total distinct words:  11285
[('other', 30790, 30814), ('first', 21509, 21533), ('many', 19333, 19357), ('new', 12499, 12523), ('system', 10159, 10183)]
refinery: 1 , 25
Next threshold =  4
Total distinct words:  9801
[('other', 38428, 38456), ('first', 26768, 26796), ('many', 24215, 24243), ('new', 15594, 15622), ('system', 12753, 12781)]
refinery: 0 , 28
Next threshold =  4
Total distinct words:  11346
[('other', 46123, 46155), ('first', 32013,

In [1]:
from pyspark.streaming import StreamingContext

# Create a queue of RDDs
rddQueue = []
for i in range(5):
    rdd = sc.parallelize([i, i, i, i, i])
    rddQueue += [rdd]
        
# Create a StreamingContext with batch interval of 3 seconds
ssc = StreamingContext(sc, 3)

ssc.checkpoint("checkpoint")

# Feed the rdd queue to a DStream
nums = ssc.queueStream(rddQueue)

# Compute the sum over a sliding window of 9 seconds for every 3 seconds
# slidingSum = nums.reduceByWindow(lambda x, y: x + y, None, 9, 3)
slidingSum = nums.reduceByWindow(lambda x, y: x + y, lambda x, y: x - y, 9, 3)

slidingSum.pprint()

ssc.start()  # Start the computation
ssc.awaitTermination(24)  # Wait for the computation to terminate
ssc.stop(False)
print ("Finished")

-------------------------------------------
Time: 2018-12-01 17:25:54
-------------------------------------------
0

-------------------------------------------
Time: 2018-12-01 17:25:57
-------------------------------------------
5

-------------------------------------------
Time: 2018-12-01 17:26:00
-------------------------------------------
15

-------------------------------------------
Time: 2018-12-01 17:26:03
-------------------------------------------
30

-------------------------------------------
Time: 2018-12-01 17:26:06
-------------------------------------------
45

-------------------------------------------
Time: 2018-12-01 17:26:09
-------------------------------------------
35

-------------------------------------------
Time: 2018-12-01 17:26:12
-------------------------------------------
20

-------------------------------------------
Time: 2018-12-01 17:26:15
-------------------------------------------

Finished


In [19]:
from pyspark.sql.functions import *


lines = spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', '9999')\
        .option('includeTimestamp', 'true')\
        .load()
        
# Split the lines into words, retaining timestamps
# split() splits each line into an array, and explode() turns the array into multiple rows
words = lines.select(explode(split(lines.value, ' ')).alias('word'),
                     lines.timestamp)

word_counts = words.groupBy('word').count()

# Start running the query 
query = word_counts\
        .writeStream\
        .outputMode('complete')\
        .format('console')\
        .option('truncate', 'false')\
        .trigger(processingTime='5 seconds') \
        .start()

query.awaitTermination(25)
query.stop()
print ("Finished")

StreamingQueryException: 'Connection refused (Connection refused)\n=== Streaming Query ===\nIdentifier: [id = dc69991f-ff8f-4983-b3ad-2a36fda8d365, runId = 42db0ddf-460c-4d98-98e8-6900ac6353d9]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {}\n\nCurrent State: INITIALIZING\nThread State: RUNNABLE'

In [10]:
from pyspark.sql.functions import *

lines = spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', '9999')\
        .option('includeTimestamp', 'true')\
        .load()
        
# Split the lines into words, retaining timestamps
# split() splits each line into an array, and explode() turns the array into multiple rows
words = lines.select(explode(split(lines.value, ' ')).alias('word'),
                     lines.timestamp)

long_words = words.filter(length(words['word'])>=3)

# Start running the query 
query = long_words\
        .writeStream\
        .outputMode('append')\
        .format('console')\
        .option('truncate', 'false')\
        .trigger(processingTime='5 seconds') \
        .start()

query.awaitTermination(25)
query.stop()
print "Finished"

Finished


In [20]:
from pyspark.sql.functions import *

lines = spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', '9999')\
        .option('includeTimestamp', 'true')\
        .load()
        
# Split the lines into words, retaining timestamps
# split() splits each line into an array, and explode() turns the array into multiple rows
words = lines.select(explode(split(lines.value, ' ')).alias('word'),
                     lines.timestamp)

windowedCounts = words.groupBy(
    window(words.timestamp, "10 seconds", "5 seconds"),
    words.word)\
    .count()

# Start running the query 
query = windowedCounts\
        .writeStream\
        .outputMode('complete')\
        .format('console')\
        .option('truncate', 'false')\
        .trigger(processingTime='5 seconds') \
        .start()

query.awaitTermination(25)
query.stop()
print ("Finished")

StreamingQueryException: 'Connection refused (Connection refused)\n=== Streaming Query ===\nIdentifier: [id = 31928b80-f171-4973-8e97-f553e56dd8a0, runId = c68a8f95-a0e7-410c-9afc-4d6137474067]\nCurrent Committed Offsets: {}\nCurrent Available Offsets: {}\n\nCurrent State: INITIALIZING\nThread State: RUNNABLE'